# RAG Evaluations


In [5]:
from llama_index.core import SimpleDirectoryReader
path = "../data/samples/"
docs = SimpleDirectoryReader(
    input_dir=path
).load_data()
len(docs)

12

In [7]:
docs[3].text

'---\ntitle: "Being Inclusive"\ndescription: "We are all responsible for creating and maintaining an inclusive environment at GitLab."\n---\n\n## What is fundamental for a successful inclusive environment\n\nWe are **all responsible** for creating and maintaining an inclusive environment at GitLab.\n\nWe all need to feel **empowered** to contribute and collaborate.\n\n## Diversity, Inclusion & Belonging at GitLab\n\nDiversity includes all the differences we each have, whether it be where we grew up, where we went to school, experiences, how we react, age, race, gender, national origin, things we can and can not see, the list goes on.\n\nInclusion is understanding or recognizing all these differences and inviting someone to be a part of things and collaborate, etc.\n\nBelonging is when you feel your insights and contributions are valued. It goes back to team members feeling they can bring their full selves to work.\n\nSee the [Diversity, Inclusion, and Belonging page]({{< ref "inclusion

### Construct a RAG retriever


In [8]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs)
query_engine = index.as_query_engine()

In [9]:
response = query_engine.query("What does it mean to be inclusive in Gitlab?")
print(response.response)

Being inclusive at GitLab means creating and maintaining an environment where all team members feel empowered to contribute and collaborate. It involves recognizing and valuing the differences among individuals, inviting everyone to participate, ensuring that all voices are heard, and making efforts to understand and leverage each team member's contributions. Inclusive behaviors include seeking input from diverse backgrounds, active listening, asking questions, addressing misunderstandings, assuming positive intent, and avoiding making assumptions based on personal characteristics. GitLab emphasizes the importance of holding each other accountable for using inclusive language, avoiding identity assumption statements, and uncovering unconscious biases to create a supportive and welcoming work environment for all team members.


## Create an Evaluation Data Set


In [10]:
from ragas.embeddings import LangchainEmbeddingsWrapper, LlamaIndexEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper, LlamaIndexLLMWrapper
from ragas.testset import TestsetGenerator
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

llama_index_llm = LlamaIndexLLMWrapper(OpenAI(model="gpt-4o"))
llama_index_embeddings = LlamaIndexEmbeddingsWrapper(OpenAIEmbedding())

generator = TestsetGenerator(llm=llama_index_llm, embedding_model=llama_index_embeddings)
dataset = generator.generate_with_llamaindex_docs(docs, testset_size=10)

df = dataset.to_pandas()
df.head()

Applying HeadlinesExtractor:   0%|          | 0/8 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/12 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/11 [00:00<?, ?it/s]

Property 'summary' already exists in node '52111d'. Skipping!
Property 'summary' already exists in node 'ed5a3c'. Skipping!
Property 'summary' already exists in node '26531b'. Skipping!


...

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Wht is GitLab's role in suporting allys?,"[--- title: ""The Ally Lab"" description: Learn ...","GitLab provides tools, resources, and learning...",single_hop_specifc_query_synthesizer
1,How can Zoom be utilized to address feedback i...,[Skills and Behaviors of allies To be an effec...,"After a meeting, someone from the group can ju...",single_hop_specifc_query_synthesizer
2,What does an ALLG Facilitator do in the contex...,[Concepts & Terms 1. Privilege: an unearned ad...,An ALLG Facilitator leads a group of 5-10 team...,single_hop_specifc_query_synthesizer
3,what equality ally strategies?,[What it means to be an ally - Take on the str...,Equality Ally Strategies are part of the essen...,single_hop_specifc_query_synthesizer
4,How can empathy and active listening enhance a...,"[<1-hop>\n\n--- title: ""The Ally Lab"" descript...",Empathy and active listening are crucial skill...,multi_hop_abstract_query_synthesizer


In [13]:
df.iloc[1]["user_input"]

'How can Zoom be utilized to address feedback in a non-defensive manner?'

In [16]:
print(df.iloc[1]["reference"])

After a meeting, someone from the group can jump on a Zoom meeting with you to explain how your actions, such as talking over someone, affected the group. This allows you to apologize, take on the feedback, and ask for tips on how to improve, demonstrating humility and a willingness to learn.


In [15]:
response = query_engine.query(df.iloc[1]["user_input"])
print(response.response)


Zoom can be utilized to address feedback in a non-defensive manner by actively listening to the feedback provided without interrupting or becoming defensive. It is important to maintain an open and receptive attitude during the conversation, allowing the person giving feedback to express their thoughts fully. Additionally, using Zoom's video feature can help convey non-verbal cues such as nodding and maintaining eye contact to show that you are engaged and taking the feedback seriously. Finally, summarizing the feedback received and asking clarifying questions can demonstrate that you value the input and are committed to understanding and addressing any concerns raised.
